In [26]:
import os
import pandas as pd

version = "v01"
main_publisher = 'OYC'
script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')

df_series = pd.read_csv(path + 'series.csv', delimiter = '|')
df_episodes = pd.read_csv(path + 'episodes.csv', delimiter = '|')
df_chapters = pd.read_csv(path + 'chapters.csv', delimiter = '|')
chapters = df_chapters['Cid'].values
df_concepts = pd.read_csv(path + 'concepts.csv', delimiter = '|')
df_precedence = pd.DataFrame()
df_concepts.shape

(663855, 3)

In [5]:
from rdflib import Graph

df_concepts = pd.DataFrame()
missing = []
for c in chapters:
    concepts = []
    path_graph = f'../Output/Graphs/{version}/{c}.ttl'
    g = Graph()
    try :
        g.parse(path_graph, format='turtle')
    except (FileNotFoundError, IOError):
        print(f'Chapter {c} file not found !')
        missing.append(c)
        continue
    concepts_query = """
            SELECT ?concept ?pr WHERE
            {
                ?ER dct:subject ?concept.
                ?concept ns1:pageRank ?pr.
            }
    """
    result = g.query(concepts_query)
    for row in result:
        concept_dict = {
            'OER' : c,
            'Concept' : str(row.concept),
            'PR' : float(str(row.pr))
        }
        concepts.append(concept_dict)
    df_concepts = pd.concat([df_concepts, pd.DataFrame(concepts)], ignore_index = True)
print(df_concepts.shape)
df_concepts.to_csv(path + 'concepts.csv', sep = '|', index = False)

Chapter 79 file not found !
Chapter 412 file not found !
Chapter 452 file not found !
Chapter 672 file not found !
Chapter 1022 file not found !
Chapter 1031 file not found !
Chapter 1040 file not found !
Chapter 1046 file not found !
Chapter 1054 file not found !
Chapter 1066 file not found !
Chapter 1075 file not found !
Chapter 1125 file not found !
Chapter 1220 file not found !
Chapter 1227 file not found !
Chapter 2176 file not found !
Chapter 2183 file not found !
Chapter 2198 file not found !
Chapter 2263 file not found !
Chapter 2269 file not found !
Chapter 2271 file not found !
Chapter 2272 file not found !
Chapter 2281 file not found !
Chapter 2289 file not found !
Chapter 2293 file not found !
Chapter 2297 file not found !
Chapter 2306 file not found !
Chapter 2307 file not found !
Chapter 2312 file not found !
Chapter 2322 file not found !
Chapter 2324 file not found !
Chapter 2326 file not found !
Chapter 2332 file not found !
Chapter 2336 file not found !
Chapter 2550 fi

In [27]:
pr_filter = 0.005
df_concepts_bis = df_concepts[df_concepts['PR'] > pr_filter]

print(len(df_concepts), '\tNUM of concepts')
print(len(df_concepts_bis), '\tNUM of concepts w filter on PR = ',pr_filter)
print(round(len(df_concepts)/len(df_concepts_bis), 2), "\t% Compression after filtering")

value_counts = df_concepts_bis['OER'].value_counts()
rec_value_mean = int(value_counts.mean())
print(rec_value_mean, '\tAVG concepts per chapter')

rec_value_median = int(value_counts.median())
print(rec_value_median, '\tMEDIAN concepts per chapter')

rec_value_max = value_counts.max()
print(rec_value_max, '\tMAX concepts per chapter')

rec_value_min = value_counts.min()
print(rec_value_min, '\tMIN concepts per chapter')

df_concepts_bis.to_csv(path + 'concepts_bis.csv', sep = '|', index = False)

663855 	NUM of concepts
16786 	NUM of concepts w filter on PR =  0.005
39.55 	% Compression after filtering
6 	AVG concepts per chapter
6 	MEDIAN concepts per chapter
31 	MAX concepts per chapter
1 	MIN concepts per chapter


In [13]:
precedences = []
for c in chapters[:-1]:
    if df_chapters[df_chapters['Cid'] == c]['Sid'].values[0] == df_chapters[df_chapters['Cid'] == c + 1]['Sid'].values[0] :
        precedence_dict = {
            'Before' : c,
            'After' : c + 1,
            'Sid' : df_chapters[df_chapters['Cid'] == c]['Sid'].values[0]
        }
        precedences.append(precedence_dict)

df_precedence = pd.DataFrame(precedences)
df_precedence.to_csv(path + 'precedences.csv', sep='|', index = False)
df_precedence.head()

,Before,After,Sid
0,0,1,0
1,1,2,0
2,2,3,0
3,3,4,0
4,4,5,0


In [14]:
precedences = []
for c in chapters[:-1]:
    c_next = c + 1
    while c_next != len(chapters) and df_chapters[df_chapters['Cid'] == c]['Sid'].values[0] == df_chapters[df_chapters['Cid'] == c_next]['Sid'].values[0]:
        precedence_dict = {
            'Before' : c,
            'After' : c_next,
            'Sid' : df_chapters[df_chapters['Cid'] == c]['Sid'].values[0]
        }
        precedences.append(precedence_dict)
        c_next += 1
df_precedence_all = pd.DataFrame(precedences)
df_precedence_all.to_csv(path + 'precedences_all.csv', sep='|', index = False)
df_precedence_all.head()

,Before,After,Sid
0,0,1,0
1,0,2,0
2,0,3,0
3,0,4,0
4,0,5,0


In [21]:
precedences = []
for c in chapters[:-1]:
    c_next = c + 1
    c_eid = df_chapters[df_chapters['Cid'] == c]['Eid'].values[0]
    c_next_eid = df_chapters[df_chapters['Cid'] == c_next]['Eid'].values[0]
    if c_eid == c_next_eid:
        precedence_dict = {
            'Before' : c,
            'After' : c_next,
            'Eid' : df_chapters[df_chapters['Cid'] == c]['Eid'].values[0]
        }
        precedences.append(precedence_dict)
        c_next += 1
        if(c_next < len(chapters)):
            print(c_next)
            c_next_eid = df_chapters[df_chapters['Cid'] == c_next]['Eid'].values[0]
df_precedence_episodes = pd.DataFrame(precedences)
df_precedence_episodes.to_csv(path + 'precedences_episodes.csv', sep='|', index = False)
df_precedence_episodes.head()

2
3
4
5
6
8
9
10
11
13
14
15
16
17
19
20
21
22
24
25
27
28
29
31
32
33
34
35
37
38
39
40
42
43
44
46
47
48
49
51
52
54
55
56
57
58
60
61
62
64
65
66
67
68
70
71
72
73
74
76
77
79
80
81
83
84
85
86
88
89
90
91
92
93
95
96
97
98
99
100
102
103
104
105
106
107
108
109
111
112
113
114
116
117
119
120
121
122
123
124
125
126
127
129
130
131
132
133
134
135
137
138
139
140
141
142
144
145
146
147
148
149
150
151
153
154
155
156
157
158
159
160
162
163
164
165
166
167
168
169
170
172
173
174
175
176
177
178
180
181
182
183
184
185
186
188
189
190
191
192
193
194
196
197
198
199
200
201
202
203
205
206
207
208
209
210
211
212
213
214
216
217
218
219
220
221
223
224
225
226
227
228
229
230
231
233
234
235
236
237
238
239
240
241
243
244
245
246
247
248
249
250
252
253
254
255
256
257
258
259
260
262
263
264
265
266
267
268
270
271
272
273
274
275
276
277
279
280
281
282
283
284
286
287
288
289
290
291
292
293
295
296
297
298
299
300
301
302
304
305
306
307
308
309
310
312
313
314
315
316
317
31

,Before,After,Eid
0,0,1,0
1,1,2,0
2,2,3,0
3,3,4,0
4,4,5,0


In [22]:
df_precedence_episodes.shape

(2097, 3)

In [17]:
precedences = []
for c in chapters[:-1]:
    c_next = c + 1
    c_eid = df_chapters[df_chapters['Cid'] == c]['Eid'].values[0]
    c_next_eid = df_chapters[df_chapters['Cid'] == c_next]['Eid'].values[0]
    c_sid = df_chapters[df_chapters['Cid'] == c]['Sid'].values[0]
    c_next_sid = df_chapters[df_chapters['Cid'] == c_next]['Sid'].values[0]
    if c_eid != c_next_eid and c_sid == c_next_sid:
        precedence_dict = {
            'Before' : c,
            'After' : c_next,
            'Sid' : df_chapters[df_chapters['Cid'] == c]['Sid'].values[0]
        }
        precedences.append(precedence_dict)
df_precedence_series = pd.DataFrame(precedences)
df_precedence_series.to_csv(path + 'precedences_series.csv', sep='|', index = False)
df_precedence_series.head()

,Before,After,Sid
0,5,6,0
1,10,11,0
2,16,17,0
3,21,22,0
4,24,25,0


In [43]:
df_precedence_series.shape

(1407, 3)